# RESTful API utilizando Flask

### O objetivo desse Notebook é apresentar uma API RESTful na maneira mais simples/enxuta possível. 
    
    
### Tratamento de erros, excessões, questões de acesso e segurança foram deixadas para futuras implementações.

# 0 - Importação das Bibliotecas/Framework

In [7]:
from flask import Flask, jsonify, request, Response
from flask_restful import Resource, Api
from flask_pymongo import PyMongo
from bson import json_util
from bson.objectid import ObjectId
import json
import pymongo

# 1 - Declaração do conteúdo do DB em formato JSON

Abaixo temos listas em formato JSON contendo informações a respeito de quatro bandas de heavy metal brasileiras, este pequeno conjunto de informações irá popular o nosso banco de dados utilizado para os testes.

In [2]:
muslist = [
  {
       "banda": "Viper",
       "cidade_origem" : "Sao Paulo",
       "locais_shows" : ["0001","0003"]
  },
  { 
      "banda": "Sepultura",
      "cidade_origem" : "Belo Horizonte",
      "locais_shows" : "0003"
  },
  {
      "banda": "Vulcano",
      "cidade_origem" : "Santos",
      "locais_shows" : "0005"
  },
  {
      "banda": "Steel Warrior",
      "cidade_origem" : "Itajai",
      "locais_shows" : "0007"
  }
]

# 2 - Conexão e Inserção de dados no MongoDB:

Optei pela utilização de um banco de dados de documentos  (ié, armazena documentos na parte do valor do armazenamento de chave-valor), mais especificamente o "NoSQL" MongoDB devido a sua popularidade e simplicidade.

Argumentações a respeito de vantagens técnicas do MongoDB (ex: sua alta disponibilidade) fogem do escopo desta simples demonstração.

In [3]:
import pymongo
client = pymongo.MongoClient("mongodb://localhost:27017/")

db = client["RESTdb"]

#muscol = db["Musicos"]
#showcol = db["Shows"]
if 'Musicos' in db.list_collection_names():
    pass
else:
    muscol = db["Musicos"]
    muscol.insert_many(muslist)
    
client.close()

# 3-  Inicialização do Flask (?)

In [8]:
app = Flask(__name__)
app.config['MONGO_URI'] = 'mongodb://localhost/RESTdb'
mongo = PyMongo(app)

# 4 - Aplicação do decorator "route":


Como se trata de uma API que faz uso da arquitetura REST ("RESTful API"), executamos uma função para cada método HTTP:

[Mais informações sobre o decorator "route" aqui](https://flask.palletsprojects.com/en/2.0.x/api/#flask.Flask.route)

In [5]:
@app.route('/Musicos', methods=['GET'])
def get_users():
    musicos = mongo.db.Musicos.find()
    response = json_util.dumps(musicos)
    return Response(response, mimetype="application/json")


@app.route('/Musicos', methods=['POST'])
def create_user():    
    banda = request.json['banda']
    cidade_origem = request.json['cidade_origem']
    locais_shows = request.json['locais_shows']

    if username and email and password:
        hashed_password = generate_password_hash(password)
        id = mongo.db.Musicos.insert(
            {'banda': banda, 'cidade_origem': cidade_origem, 'locais_shows': locais_shows})
        response = jsonify({
            '_id': str(id),
            'bandas': bandas,
            'cidade_origem': cidade_origem,
            'locais_shows': locais_shows
        })
        response.status_code = 201
        return response
    else:
        return 'Nao Encontrado (Not Found)'
    
    
@app.route('/Musicos/<_id>', methods=['PUT'])
def update_user(_id):
    banda = request.json['banda']
    cidade_origem = request.json['cidade_origem']
    locais_shows = request.json['locais_shows']
    if banda and cidade_origem and locais_shows and _id:
        mongo.db.Musicos.update_one(
            {'_id': ObjectId(_id['$oid']) if '$oid' in _id else ObjectId(_id)}, {'$set': { \
                'banda': banda, 'cidade_origem': cidade_origem, 'locais_shows': locais_shows}})
        response = jsonify({'message': 'Musico' + _id + 'Updated Successfuly'})
        response.status_code = 200
        return response
    else:
      return 'Nao Encontrado (Not Found)' 
    
      
@app.route('/Musicos/<id>', methods=['DELETE'])
def delete_user(id):
    mongo.db.Musicos.delete_one({'_id': ObjectId(id)})
    response = jsonify({'message': 'Musico' + id + ' Deleted Successfully'})
    response.status_code = 200
    return response



# 5 - Voilà! 

Depois de rodar, basta acessar o localhost por meio de ferramentas como o [CURL](https://curl.se/) ou [Postman](https://www.postman.com/downloads/).



In [6]:
if __name__ == '__main__':
    app.run() 

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
